# 环境设置

In [3]:
import dlib
import cv2
import numpy as np
import os
from PIL import Image
import json
from numpy.linalg import inv
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.font_manager import FontProperties
import pickle
from IPython.display import display, clear_output
import ipywidgets as widgets
# 设置中文字体（全局设置）

font_path = r"C:\Users\34568\simhei.ttf"  #字体路径
chinese_font = FontProperties(fname=font_path)
mpl.rcParams["font.family"] = chinese_font.get_name()  # 设置字体名称
mpl.rcParams["axes.unicode_minus"] = False

base_working_dir = r'C:\Users\34568\Desktop\GlassesGAN'
repo_name_dir = 'GlassesGAN_release'
msvc_path = r"C:\Program Files (x86)\Microsoft Visual Studio\2019\BuildTools\VC\Tools\MSVC\14.29.30133\bin\Hostx64\x64"
os.environ["PATH"] = msvc_path + ";" + os.environ.get("PATH", "")
predictor_path = r"C:\Users\34568\Desktop\GlassesGAN\GlassesGAN_release\encoder4editing\shape_predictor_68_face_landmarks.dat"
image_path = r"imgtest/002.jpg"  

# 获取人脸关键参数

In [4]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

faces = detector(gray)

for face in faces:
    landmarks = predictor(gray, face)
    
    # 眼睛中心点
    left_eye_center = (landmarks.part(36).x + landmarks.part(39).x) / 2, (landmarks.part(36).y + landmarks.part(39).y) / 2
    right_eye_center = (landmarks.part(42).x + landmarks.part(45).x) / 2, (landmarks.part(42).y + landmarks.part(45).y) / 2
    
    # 眼睛间距
    eye_distance = np.linalg.norm(np.array(left_eye_center) - np.array(right_eye_center))  # 眼睛间距
    
    # 脸宽
    left_cheek = (landmarks.part(0).x, landmarks.part(0).y)
    right_cheek = (landmarks.part(16).x, landmarks.part(16).y)
    face_width = np.linalg.norm(np.array(left_cheek) - np.array(right_cheek))  # 脸宽
    
    # 眼睛高度（垂直方向）
    eye_height = np.abs(landmarks.part(37).y - landmarks.part(41).y)  # 左眼上下高度差
    
    # 鼻梁宽度和高度
    nose_width = np.linalg.norm(np.array((landmarks.part(31).x, landmarks.part(31).y)) - np.array((landmarks.part(35).x, landmarks.part(35).y)))  # 鼻梁宽度
    nose_height = np.abs(landmarks.part(27).y - landmarks.part(33).y)  # 鼻梁高度
    
    # 下巴高度
    chin_height = np.abs(landmarks.part(8).y - landmarks.part(33).y)  # 下巴高度（从下巴到鼻尖）
    
    # 面部高度
    face_height = np.abs(landmarks.part(27).y - landmarks.part(8).y)  # 从鼻梁到下巴的高度
    
    # 计算比值
    eye_distance_ratio = eye_distance / face_width  # 眼睛间距与脸宽比值
    eye_height_ratio = eye_height / eye_distance  # 眼睛上下高度与眼睛间距比值
    nose_width_ratio = nose_width / face_width  # 鼻梁宽度与脸宽比值
    face_ratio = face_height / face_width  # 脸型的宽高比
    chin_to_nose_ratio = chin_height / nose_height  # 下巴高度与鼻梁高度比值
    
    # 输出量化参数
    print(f"眼睛间距: {eye_distance:.2f} 像素, 眼睛间距与脸宽比值: {eye_distance_ratio:.2f}")
    print(f"眼睛高度: {eye_height:.2f} 像素, 眼睛高度与眼睛间距比值: {eye_height_ratio:.2f}")
    print(f"鼻梁宽度: {nose_width:.2f} 像素, 鼻梁宽度与脸宽比值: {nose_width_ratio:.2f}")
    print(f"下巴高度: {chin_height:.2f} 像素, 下巴高度与鼻梁高度比值: {chin_to_nose_ratio:.2f}")
    print(f"面部高度: {face_height:.2f} 像素, 面部宽高比: {face_ratio:.2f}")

眼睛间距: 42.03 像素, 眼睛间距与脸宽比值: 0.46
眼睛高度: 5.00 像素, 眼睛高度与眼睛间距比值: 0.12
鼻梁宽度: 16.03 像素, 鼻梁宽度与脸宽比值: 0.18
下巴高度: 39.00 像素, 下巴高度与鼻梁高度比值: 1.22
面部高度: 71.00 像素, 面部宽高比: 0.78


# 图像初始化

In [5]:
# 切换到项目目录并导入核心类
os.chdir(f'{base_working_dir}/{repo_name_dir}')
from glasses_vton_inference import glasses_vton_inference

# 初始化参数
glasses_option = 'RG'
chosen_deeplab_epoch = 19
fitted_pca_fp = f'{base_working_dir}/fitted_pca_celebhq_dataset_results.joblib'
ave_add_glasses_diff_fp = f'{base_working_dir}/aveglassesdiff_celebhq_dataset_results_RG.npy'
resume_model_ckpt = f'{base_working_dir}/deeplab_epoch_{chosen_deeplab_epoch}.pth'

# 初始化 GlassesVton 类实例
gvton = glasses_vton_inference(
    fitted_pca_fp=fitted_pca_fp,
    ave_add_glasses_diff_fp=ave_add_glasses_diff_fp,
    base_working_dir=base_working_dir,
    temp_save_folder=f'{base_working_dir}/tempfolder',
    resume_model_ckpt=resume_model_ckpt,
    deeplab_script_location=f'{base_working_dir}/GlassesGAN_release/datasetGAN_release/datasetGAN',
    chosen_deeplab_epoch=chosen_deeplab_epoch,
    load_loc=base_working_dir,
    use_full_glasses_or_frames_mask='frames',
    run_tests=False,
    outer_dilation_factor=5,
    outer_blur_factor=12,
    inner_blur_factor=5,
    ideal_avg_glasses_frame_area=0.020,
    auto_clean=True
)
input_image, result_image, base_latent = gvton.embed_input_image(image_path, show_plots=False)
start_latent, edit_image, _ = gvton.add_avg_glasses(input_image, base_latent, base_bias=1, show_plots=False, auto_pick_bias=True, return_blended_image=True)

Loading e4e over the pSp framework from checkpoint: pretrained_models/e4e_ffhq_encode.pt
Model successfully loaded!
Model checkpoint: C:\Users\34568\Desktop\GlassesGAN/deeplab_epoch_19.pth (valid)
DeepLab script: C:\Users\34568\Desktop\GlassesGAN/GlassesGAN_release/datasetGAN_release/datasetGAN\test_deeplab_cross_validation.py (valid)

Removing temporary folder

Copying model to temporary working directory
Aligned image has shape: (1024, 1024)
Inference took 0.6839 seconds.

Saving image and fake groundtruth maps to temporary file location


0it [00:00, ?it/s]

Resizing image to (512,512,3)


1it [00:00,  1.89it/s]

Resizing image to (512,512,3)


...


Running deeplab
Opt {'exp_dir': 'C:\\Users\\34568\\Desktop\\GlassesGAN/tempfolder/model', 'batch_size': 64, 'category': 'face', 'debug': False, 'dim': [512, 512, 5088], 'deeplab_res': 512, 'number_class': 4, 'testing_data_number_class': 4, 'max_training': 7, 'stylegan_ver': '1', 'annotation_data_from_w': False, 'annotation_mask_path': './custom_data/annotation/training_data', 'testing_path': 'C:\\Users\\34568\\Desktop\\GlassesGAN/tempfolder/input_images', 'average_latent': './custom_data/training_latent/avg_latent_stylegan1.npy', 'annotation_image_latent_path': './custom_data/training_latent/latent_stylegan1.npy', 'stylegan_checkpoint': './checkpoints/stylegan_pretrain/karras2019stylegan-ffhq-1024x1024_old_serialization.pt', 'model_num': 10, 'upsample_mode': 'bilinear'}
args Namespace(exp='C:\\Users\\34568\\Desktop\\GlassesGAN/tempfolder/face_34.json', resume='C:\\Users\\34568\\Desktop\\GlassesGAN/tempfolder/model', cross_validate=False, validation_number=0, chosen_deeplab_epoch=19)
R

0it [00:00, ?it/s]

Resizing image to (512,512,3)


1it [00:00,  2.33it/s]



Running deeplab
Opt {'exp_dir': 'C:\\Users\\34568\\Desktop\\GlassesGAN/tempfolder/model', 'batch_size': 64, 'category': 'face', 'debug': False, 'dim': [512, 512, 5088], 'deeplab_res': 512, 'number_class': 4, 'testing_data_number_class': 4, 'max_training': 7, 'stylegan_ver': '1', 'annotation_data_from_w': False, 'annotation_mask_path': './custom_data/annotation/training_data', 'testing_path': 'C:\\Users\\34568\\Desktop\\GlassesGAN/tempfolder/input_images', 'average_latent': './custom_data/training_latent/avg_latent_stylegan1.npy', 'annotation_image_latent_path': './custom_data/training_latent/latent_stylegan1.npy', 'stylegan_checkpoint': './checkpoints/stylegan_pretrain/karras2019stylegan-ffhq-1024x1024_old_serialization.pt', 'model_num': 10, 'upsample_mode': 'bilinear'}
args Namespace(exp='C:\\Users\\34568\\Desktop\\GlassesGAN/tempfolder/face_34.json', resume='C:\\Users\\34568\\Desktop\\GlassesGAN/tempfolder/model', cross_validate=False, validation_number=0, chosen_deeplab_epoch=19)
R

# 基于上下文老虎机的推荐

In [6]:
class GlassesRecommender:
    """眼镜推荐系统（使用已知人脸参数）"""
    def __init__(self, param_ranges=None, alpha=0.2, load_path=None):
        self.param_ranges = param_ranges or [
            (-5.0, 10.0),    # Size
            (-10.0, 4.0),    # Height
            (-4.0, 20.0),    # Squareness
            (-20.0, 20.0),   # Round_Shrink
            (-15.0, 20.0),   # Cateye
            (-10.0, 10.0)    # Thicken
        ]
        self.param_names = ["Size", "Height", "Squareness", "Round_Shrink", "Cateye", "Thicken"]
        self.alpha = alpha
        self.context_dim = 5
        self.param_dim = 6
        self.expanded_dim = self.context_dim + self.param_dim + self.context_dim * self.param_dim
        self.A = np.eye(self.expanded_dim)
        self.b = np.zeros(self.expanded_dim)
        self.history = []
        if load_path and os.path.exists(load_path):
            self.load_model(load_path)
    
    def build_feature_vector(self, face_features, glass_params):
        base_features = np.concatenate([face_features, glass_params])
        interaction_terms = np.outer(face_features, glass_params).flatten()
        return np.concatenate([base_features, interaction_terms])
    
    def generate_candidate_params(self, n_candidates=50, strategy='lhs'):
        if strategy == 'lhs':
            candidates = np.zeros((n_candidates, self.param_dim))
            for i, (low, high) in enumerate(self.param_ranges):
                segments = np.linspace(0, 1, n_candidates + 1)
                points = np.random.uniform(segments[:-1], segments[1:])
                np.random.shuffle(points)
                candidates[:, i] = low + points * (high - low)
            return candidates
        else:
            return np.array([[np.random.uniform(low, high) for (low, high) in self.param_ranges] for _ in range(n_candidates)])
    
    def recommend(self, face_features, n_candidates=50, return_all=False):
        candidate_params = self.generate_candidate_params(n_candidates)
        scores = []
        for params in candidate_params:
            feature_vector = self.build_feature_vector(face_features, params)
            A_inv = inv(self.A)
            theta = A_inv @ self.b
            expected_reward = theta.dot(feature_vector)
            uncertainty = self.alpha * np.sqrt(feature_vector.dot(A_inv).dot(feature_vector))
            scores.append(expected_reward + uncertainty)
        best_idx = np.argmax(scores)
        return (candidate_params[best_idx], candidate_params, scores) if return_all else candidate_params[best_idx]
    
    def update(self, face_features, glass_params, reward):
        feature_vector = self.build_feature_vector(face_features, glass_params)
        self.A += np.outer(feature_vector, feature_vector)
        self.b += reward * feature_vector
        self.history.append({'face_features': face_features, 'glass_params': glass_params, 'reward': reward})
        print(f"模型已更新: 奖励={reward}")
    
    def save_model(self, path):
        model_data = {'A': self.A, 'b': self.b, 'history': self.history, 'param_ranges': self.param_ranges, 'alpha': self.alpha}
        with open(path, 'wb') as f:
            pickle.dump(model_data, f)
        print(f"模型已保存到: {path}")
    
    def load_model(self, path):
        try:
            with open(path, 'rb') as f:
                model_data = pickle.load(f)
            self.A = model_data['A']
            self.b = model_data['b']
            self.history = model_data['history']
            print(f"模型加载完成")
        except Exception as e:
            print(f"加载模型失败: {e}")


# 生成眼镜图像
def generate_and_show_image(glass_params, start_latent, gvton, input_image=None, title="推荐眼镜"):
    Size, Height, Squareness, Round_Shrink, Cateye, Thicken = glass_params
    num_pcs = 6
    latent = start_latent.copy()
    for PC_num, PC_bias in enumerate((Size, Height, Squareness, Round_Shrink, Cateye, Thicken)):
        run_gen = True if PC_num == (num_pcs - 1) else False
        img, latent = gvton.e4e.run_gen_add_pc_direction_bias(
            start_latent=latent, fitted_pca=gvton.fitted_pca, bias=PC_bias, PC_num=PC_num, run_gen=run_gen
        )
    if input_image is not None:
        blends, _, _ = gvton.blend_in_edits(edits=[img], input_image=input_image)
        result_image = blends[0]
    else:
        result_image = img
    plt.figure(figsize=(5, 5))
    plt.imshow(result_image)
    plt.title(title, fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    return result_image

def create_recommendation_interface(recommender, gvton, start_latent, input_image=None, 
                                   eye_distance_ratio=None, eye_height_ratio=None, 
                                   nose_width_ratio=None, face_ratio=None, chin_to_nose_ratio=None):
    output = widgets.Output()
    save_output = widgets.Output()
    
    # 检查参数是否有效
    required_params = [
        eye_distance_ratio, eye_height_ratio, nose_width_ratio,
        face_ratio, chin_to_nose_ratio
    ]
    
    if any(param is None for param in required_params):
        with output:
            print("错误：人脸特征参数存在未赋值的变量！")
            print("请确保以下参数均已正确赋值：")
            print("  eye_distance_ratio, eye_height_ratio, nose_width_ratio")
            print("  face_ratio, chin_to_nose_ratio")
        return  # 终止函数，避免后续错误
    
    # 显示已知参数
    with output:
        print("已加载人脸特征参数:")
        print(f"  眼睛间距/脸宽: {eye_distance_ratio:.2f}")
        print(f"  眼睛高度/眼睛间距: {eye_height_ratio:.2f}")
        print(f"  鼻梁宽度/脸宽: {nose_width_ratio:.2f}")
        print(f"  脸高/脸宽: {face_ratio:.2f}")
        print(f"  下巴高度/鼻梁高度: {chin_to_nose_ratio:.2f}")
    
    # 推荐按钮
    recommend_btn = widgets.Button(description='获取推荐', layout=widgets.Layout(width='150px'))
    
    rating_buttons = [
        widgets.Button(
            description='非常不喜欢',
            layout=widgets.Layout(width='120px'),
            style={'button_color': '#FF6B6B'},  # 深红色
            tooltip='奖励值: -0.3'
        ),
        widgets.Button(
            description='不喜欢',
            layout=widgets.Layout(width='120px'),
            style={'button_color': '#FFD166'},  # 浅红色
            tooltip='奖励值: -0.1'
        ),
        widgets.Button(
            description='一般',
            layout=widgets.Layout(width='120px'),
            style={'button_color': '#FFFFFF', 'text_color': '#000000'},  # 白色（黑字）
            tooltip='奖励值: 0'
        ),
        widgets.Button(
            description='喜欢',
            layout=widgets.Layout(width='120px'),
            style={'button_color': '#66D2A0'},  # 浅绿色
            tooltip='奖励值: 0.1'
        ),
        widgets.Button(
            description='非常喜欢',
            layout=widgets.Layout(width='120px'),
            style={'button_color': '#06D6A0'},  # 深绿色
            tooltip='奖励值: 0.3'
        )
    ]
    
    # 奖励值映射（按钮索引对应奖励值）
    reward_mapping = [-0.3, -0.1, 0, 0.1, 0.3]
    
    # 眼镜参数微调滑块
    param_sliders = [
        widgets.FloatSlider(
            min=low, max=high, step=0.5, value=0,
            description=name, layout=widgets.Layout(width='500px')
        )
        for name, (low, high) in zip(recommender.param_names, recommender.param_ranges)
    ]
    
    # 微调按钮
    apply_btn = widgets.Button(description='应用微调')
    regenerate_btn = widgets.Button(description='重新生成图像')
    
    # 模型保存控件
    save_path = widgets.Text(value='glasses_model.pkl', description='保存路径:')
    save_btn = widgets.Button(description='保存模型')
    
    # 状态变量
    current_face_features = np.array([
        eye_distance_ratio, eye_height_ratio, nose_width_ratio, face_ratio, chin_to_nose_ratio
    ])
    current_params = None
    current_score = None
    current_image = None
    
    def on_recommend_click(b):
        nonlocal current_params, current_score, current_image
        
        # 基于已知参数获取推荐
        best_params, all_params, all_scores = recommender.recommend(
            current_face_features, n_candidates=50, return_all=True
        )
        current_params = best_params
        current_score = all_scores[np.argmax(all_scores)]
        
        # 更新微调滑块
        for i, s in enumerate(param_sliders):
            s.value = best_params[i]
        
        # 显示结果
        with output:
            clear_output(wait=True)
            print(f"推荐参数 (匹配度: {current_score:.4f}):")
            for name, val in zip(recommender.param_names, best_params):
                print(f"  {name}: {val:.2f}")
            
            # 生成图像
            print("\n生成推荐眼镜图像:")
            current_image = generate_and_show_image(
                best_params, start_latent, gvton, input_image,
                title=f"推荐眼镜 (匹配度: {current_score:.4f})"
            )
            
            # 显示交互按钮
            display(widgets.Label("请评价推荐结果:"))
            display(widgets.HBox(rating_buttons))  # 显示5个评分按钮
            display(widgets.Label("微调参数:"))
            display(widgets.VBox(param_sliders))
            display(widgets.HBox([apply_btn, regenerate_btn]))
    
    def on_rating_click(b):
        nonlocal current_params
        if current_params is not None:
            # 获取点击的按钮索引，映射到奖励值
            btn_index = rating_buttons.index(b)
            reward = reward_mapping[btn_index]
            # 更新模型
            recommender.update(current_face_features, current_params, reward)
            # 重新推荐
            with output:
                print(f"\n已收到评价: {b.description} (奖励值: {reward})，正在更新推荐...")
            on_recommend_click(None)
    
    def on_apply_click(b):
        nonlocal current_params, current_score, current_image
        if current_face_features is None:
            return
        
        # 获取微调后的参数
        tweaked_params = np.array([s.value for s in param_sliders])
        
        # 计算新分数
        feature_vector = recommender.build_feature_vector(current_face_features, tweaked_params)
        A_inv = inv(recommender.A)
        theta = A_inv @ recommender.b
        current_score = theta.dot(feature_vector)
        current_params = tweaked_params
        
        # 显示微调结果
        with output:
            clear_output(wait=True)
            print(f"微调后的参数 (匹配度: {current_score:.4f}):")
            for name, val in zip(recommender.param_names, tweaked_params):
                print(f"  {name}: {val:.2f}")
            
            # 生成新图像
            print("\n生成微调后的眼镜图像:")
            current_image = generate_and_show_image(
                tweaked_params, start_latent, gvton, input_image,
                title=f"微调眼镜 (匹配度: {current_score:.4f})"
            )
            
            # 显示交互按钮
            display(widgets.Label("请评价微调结果:"))
            display(widgets.HBox(rating_buttons))
            display(widgets.Label("微调参数:"))
            display(widgets.VBox(param_sliders))
            display(widgets.HBox([apply_btn, regenerate_btn]))
    
    def on_regenerate_click(b):
        nonlocal current_image
        if current_params is not None:
            with output:
                clear_output(wait=True)
                print("重新生成图像:")
                current_image = generate_and_show_image(
                    current_params, start_latent, gvton, input_image,
                    title=f"眼镜图像 (匹配度: {current_score:.4f})"
                )
                display(widgets.Label("请评价图像:"))
                display(widgets.HBox(rating_buttons))
                display(widgets.Label("微调参数:"))
                display(widgets.VBox(param_sliders))
                display(widgets.HBox([apply_btn, regenerate_btn]))
    
    def on_save_click(b):
        with save_output:
            clear_output()
            try:
                recommender.save_model(save_path.value)
            except Exception as e:
                print(f"保存失败: {e}")
    
    # 绑定事件
    recommend_btn.on_click(on_recommend_click)
    for btn in rating_buttons:
        btn.on_click(on_rating_click)  # 绑定评分按钮点击事件
    apply_btn.on_click(on_apply_click)
    regenerate_btn.on_click(on_regenerate_click)
    save_btn.on_click(on_save_click)
    
    # 界面布局
    input_box = widgets.VBox([
        recommend_btn
    ])
    
    layout = widgets.VBox([
        widgets.HBox([input_box, output]),
        widgets.HBox([save_path, save_btn, save_output])
    ])
    
    display(layout)

In [8]:
# 主程序
%matplotlib inline
if __name__ == "__main__":
    # 创建推荐器
    recommender = GlassesRecommender(alpha=0.2)
    

    create_recommendation_interface(
         recommender=recommender,
         gvton=gvton,
         start_latent=start_latent,
         input_image=input_image,
         eye_distance_ratio=eye_distance_ratio,
         eye_height_ratio=eye_height_ratio,
         nose_width_ratio=nose_width_ratio,
         face_ratio=face_ratio,
         chin_to_nose_ratio=chin_to_nose_ratio
     )


In [9]:
start_latent

array([[[ 0.25393903,  0.30567357,  0.9901595 , ...,  0.07266595,
         -0.06027882,  0.656972  ],
        [ 0.21414018,  0.2364703 ,  0.15278353, ..., -0.13440952,
          0.24067889,  0.39202434],
        [ 0.80745035, -0.14258836,  0.04536586, ...,  0.16175488,
          0.07013719,  0.14838156],
        ...,
        [ 0.31148452,  0.4283461 ,  1.0234812 , ...,  0.0764092 ,
          0.03925574,  0.92301476],
        [ 0.25456086,  0.31119934,  0.9936857 , ...,  0.07522241,
         -0.06113189,  0.66009414],
        [ 0.09574604,  0.9423585 ,  1.4009855 , ...,  0.12040075,
         -0.15882038,  0.75650615]]], dtype=float32)